# IN_SYS - SW12 Exercise 2/2

## YOLO fine-tuning for racoons¶

In [1]:
import os
from ultralytics import YOLO
from pathlib import Path

In [2]:
# Get the current folder
current_folder = Path().resolve()

# Combine the folder path with the yolo file name
file_name = 'yolo11s.pt'
file_path = current_folder / file_name

print(f"The full path to the file is: {file_path}")
# Load the YOLOv11 model
model = YOLO(file_path)

The full path to the file is: /Users/lzrvc/Documents/Artificial Intelligence/My Solutions and Projects/2025 10 - Intelligent Systems at HSLU/Course material 2025/SW 12/yolo11s.pt


In [3]:
# Step 1: Define paths and setup
# Clone the raccoon dataset repository (if not already downloaded)
if not os.path.exists("raccoon_dataset"):
    os.system("git clone https://github.com/datitran/raccoon_dataset.git")

Cloning into 'raccoon_dataset'...


In [5]:
# Ensure the raccoon dataset is in YOLO format
# Organize images and labels into a structure compatible with YOLO
dataset_path = "raccoon_dataset"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

# Rename annotations to labels
if os.path.exists("raccoon_dataset/annotations"):
    if os.path.exists("raccoon_dataset/labels"):
        # If labels exists but is empty, remove it
        if not os.listdir("raccoon_dataset/labels"):
            os.rmdir("raccoon_dataset/labels")
    
    os.rename("raccoon_dataset/annotations", "raccoon_dataset/labels")

In [6]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_yolo():
    xml_dir = "raccoon_dataset/labels"
    images_dir = "raccoon_dataset/images"
    
    # Get all XML files
    xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
    
    if not xml_files:
        print("❌ No XML files found in labels folder!")
        return False
    
    print(f"🔍 Found {len(xml_files)} XML annotation files")
    
    converted = 0
    errors = 0
    
    for xml_file in xml_files:
        xml_path = os.path.join(xml_dir, xml_file)
        
        try:
            # Parse XML
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            # Get image dimensions
            size = root.find('size')
            img_width = int(size.find('width').text)
            img_height = int(size.find('height').text)
            
            # Create YOLO .txt file
            txt_filename = xml_file.replace('.xml', '.txt')
            txt_path = os.path.join(xml_dir, txt_filename)
            
            with open(txt_path, 'w') as f:
                # Find all objects in XML
                for obj in root.findall('object'):
                    # Get class name (assuming 'raccoon')
                    class_name = obj.find('name').text
                    class_id = 0  # We only have raccoons
                    
                    # Get bounding box
                    bndbox = obj.find('bndbox')
                    xmin = float(bndbox.find('xmin').text)
                    ymin = float(bndbox.find('ymin').text)
                    xmax = float(bndbox.find('xmax').text)
                    ymax = float(bndbox.find('ymax').text)
                    
                    # Convert to YOLO format (normalized center coordinates)
                    x_center = ((xmin + xmax) / 2) / img_width
                    y_center = ((ymin + ymax) / 2) / img_height
                    width = (xmax - xmin) / img_width
                    height = (ymax - ymin) / img_height
                    
                    # Write to file
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
            
            converted += 1
            print(f"   ✅ Converted: {xml_file} → {txt_filename}")
            
        except Exception as e:
            errors += 1
            print(f"   ❌ Failed to convert {xml_file}: {str(e)}")
    
    print(f"\n📊 Conversion complete:")
    print(f"   ✅ Successfully converted: {converted} files")
    print(f"   ❌ Errors: {errors} files")
    
    # Show sample of converted file
    if converted > 0:
        txt_files = [f for f in os.listdir(xml_dir) if f.endswith('.txt')]
        if txt_files:
            sample = os.path.join(xml_dir, txt_files[0])
            with open(sample, 'r') as f:
                print(f"\n📝 Sample YOLO format:")
                print(f"   {f.read().strip()}")
    
    return converted > 0

# Run conversion
convert_xml_to_yolo()

🔍 Found 200 XML annotation files
   ✅ Converted: raccoon-129.xml → raccoon-129.txt
   ✅ Converted: raccoon-3.xml → raccoon-3.txt
   ✅ Converted: raccoon-115.xml → raccoon-115.txt
   ✅ Converted: raccoon-101.xml → raccoon-101.txt
   ✅ Converted: raccoon-22.xml → raccoon-22.txt
   ✅ Converted: raccoon-36.xml → raccoon-36.txt
   ✅ Converted: raccoon-37.xml → raccoon-37.txt
   ✅ Converted: raccoon-23.xml → raccoon-23.txt
   ✅ Converted: raccoon-100.xml → raccoon-100.txt
   ✅ Converted: raccoon-114.xml → raccoon-114.txt
   ✅ Converted: raccoon-128.xml → raccoon-128.txt
   ✅ Converted: raccoon-2.xml → raccoon-2.txt
   ✅ Converted: raccoon-102.xml → raccoon-102.txt
   ✅ Converted: raccoon-116.xml → raccoon-116.txt
   ✅ Converted: raccoon-35.xml → raccoon-35.txt
   ✅ Converted: raccoon-21.xml → raccoon-21.txt
   ✅ Converted: raccoon-20.xml → raccoon-20.txt
   ✅ Converted: raccoon-34.xml → raccoon-34.txt
   ✅ Converted: raccoon-117.xml → raccoon-117.txt
   ✅ Converted: raccoon-103.xml → raccoon

True

In [7]:
# Step 2: Create a YOLO Dataset Configuration File
# The .yaml file specifies the dataset paths and class information
yaml_content = """
path: raccoon_dataset  # Root directory for dataset
train: images          # Training images folder
val: images            # Validation images folder
names:
  0: raccoon           # Define the classes in the dataset
"""
with open("raccoon.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

In [8]:
# Step 3: Load the YOLO model and fine-tune it
model = YOLO(file_path)  # Load the pre-trained YOLO model

# Train the model with the Raccoon Dataset
model.train(
    data="raccoon.yaml",  # Path to the dataset config file
    epochs=10,            # Number of training epochs (adjust as needed)
    imgsz=640,            # Image size
    batch=32,              # Batch size (adjust based on your hardware)
    workers=8,            # Number of CPU workers for data loading
    project="raccoon_training",  # Save results in this folder
    name="yolo11s_racoons_fine_tuned",       # Subfolder for this training run
    val=True,  # Ensure validation is enabled
    split=0.2,  # Automatically split 20% for validation
    save=True,
    save_period=1,
    verbose=False
)

New https://pypi.org/project/ultralytics/8.3.234 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.233 🚀 Python-3.12.8 torch-2.9.1 CPU (Apple M1 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=raccoon.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/Users/lzrvc/Documents/Artificial Intelligence/My Solutions and Projects/2025 10 - Intelligent Systems at HSLU/Course material 2025/S

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x17d9ebe90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [9]:
# Step 4: Validate and Test
# Evaluate the trained model to see its performance
metrics = model.val(verbose=False)  # No need to specify data - uses training config
print(metrics)

# Step 5: Save the Fine-Tuned Model
model.save("yolo_racoon_finetuned.pt")

Ultralytics 8.3.233 🚀 Python-3.12.8 torch-2.9.1 CPU (Apple M1 Pro)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 264.3±181.2 MB/s, size: 79.6 KB)
val: Scanning /Users/lzrvc/Documents/Artificial Intelligence/My Solutions and Projects/2025 10 - Intelligent Systems at HSLU/Course material 2025/SW 12/raccoon_dataset/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200 590.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 4.6s/it 59.3s5.6ss
                   all        200        217      0.957      0.922      0.963      0.638
Speed: 0.4ms preprocess, 290.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /Users/lzrvc/Documents/Artificial Intelligence/My Solutions and Projects/2025 10 - Intelligent Systems at HSLU/Course material 2025/SW 12/runs/detect/val
ultralytics.utils.metrics